In [121]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [122]:
#now mount to drive and set the working directory to our shared drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read in climate data

In [123]:
data_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/preprocessing/climate_subset_groupby/'
os.chdir(data_path)

In [124]:
scenarios = dict.fromkeys(os.listdir())
variables = ['pr', 'huss', 'hurs', 'ps', 'tas', 'sfcwind', 'rlds', 'rsds','tasmin', 'tasmax']

In [125]:
for scenario in scenarios.keys():
  scenario_path = data_path + scenario
  os.chdir(scenario_path)
  scenarios[scenario] = {}
  for variable in variables:
    variable_path = scenario_path +'/' + scenario + '_' + variable + '.csv'
    df = pd.read_csv(variable_path)
    scenarios[scenario][variable] = df

In [126]:
scenarios['ssp126']['ps']

,lat,lon,growing_season,ps_min,ps_percentile_25,ps_percentile_50,ps_percentile_75,ps_max,scenario
0,-16.75,35.25,1.0,98752.890,99117.0375,99276.7650,99483.23025,100016.730,ssp126
1,-16.75,35.25,2.0,98930.470,99209.3700,99381.4300,99577.98400,100110.600,ssp126
2,-16.75,35.25,3.0,98676.980,99047.7400,99254.0700,99484.81000,99812.125,ssp126
3,-16.75,35.25,4.0,98557.950,99186.4800,99348.1900,99504.70000,100139.320,ssp126
4,-16.75,35.25,5.0,98365.800,99238.5450,99498.0900,99706.82375,100624.820,ssp126
...,...,...,...,...,...,...,...,...,...
3310,-9.75,34.25,81.0,94216.310,94590.9050,94689.0155,94777.36500,95078.950,ssp126
3311,-9.75,34.25,82.0,94439.660,94656.8700,94737.1600,94847.30000,95116.270,ssp126
3312,-9.75,34.25,83.0,94366.060,94642.6500,94730.2800,94811.38000,95050.840,ssp126
3313,-9.75,34.25,84.0,94445.530,94645.3750,94718.8800,94800.23000,95121.710,ssp126


# Read in yield data

In [138]:
yield_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/preprocessing/all_data/yields/'
os.chdir(yield_path)

In [139]:
files = dict.fromkeys(os.listdir(), None)
files

{'maize.csv': None, 'maize_major.csv': None, 'maize_second.csv': None}

In [140]:
maize = pd.read_csv(yield_path + 'maize.csv', index_col = 0)
maize_major = pd.read_csv(yield_path + 'maize_major.csv', index_col = 0)
maize_second = pd.read_csv(yield_path + 'maize_second.csv', index_col = 0)

In [141]:
maize_csvs = [maize, maize_major, maize_second]

# Merge X features and yield data

In [131]:
#set some early parameters which will help with merging
variables = ['pr', 'huss', 'hurs', 'ps', 'tas', 'sfcwind', 'rlds', 'rsds','tasmin', 'tasmax']
starting_variable = 'ps'
variables.remove(starting_variable)

## Historical

## get X features in one df

In [132]:
#get all x features in one df
scenario = 'historical'
historical_df = scenarios[scenario][starting_variable]
for variable in variables:
  historical_df = historical_df.merge(scenarios[scenario][variable].drop('scenario', axis=1), on = ['lat', 'lon', 'growing_season'], how = 'left')
historical_df
print(historical_df.isna().sum())

lat                      0
lon                      0
growing_season           0
ps_min                   0
ps_percentile_25         0
ps_percentile_50         0
ps_percentile_75         0
ps_max                   0
scenario                 0
pr_min                   0
pr_percentile_25         0
pr_percentile_50         0
pr_percentile_75         0
pr_max                   0
huss_min                 0
huss_percentile_25       0
huss_percentile_50       0
huss_percentile_75       0
huss_max                 0
hurs_min                 0
hurs_percentile_25       0
hurs_percentile_50       0
hurs_percentile_75       0
hurs_max                 0
tas_min                  0
tas_percentile_25        0
tas_percentile_50        0
tas_percentile_75        0
tas_max                  0
tas_gdd                  0
sfcwind_min              0
sfcwind_percentile_25    0
sfcwind_percentile_50    0
sfcwind_percentile_75    0
sfcwind_max              0
rlds_min                 0
rlds_percentile_25       0
r

## merge yield data

In [ ]:
#make mapping of years growing seasons
years = list(range(1982, 2015, 1))
growing_seasons = list(range(1,34, 1))
mapping = pd.DataFrame(columns = ['year', 'growing_season'])
mapping['year'] = years
mapping['growing_season'] = growing_seasons
mapping

In [142]:
#merge to add growing_season mapping
for i in range(0, len(maize_csvs)):
  maize_csvs[i] = maize_csvs[i].merge(mapping, how='left', on = 'year') #merge growing seasons
  crop_type = maize_csvs[i]['crop_type'][0] #save crop_type for columns name
  maize_csvs[i] = maize_csvs[i].drop(['year', 'crop_type'], axis =1 ) #drop columns we dont need
  #rename columns to match
  maize_csvs[i] = maize_csvs[i].rename(columns = {'latitude':'lat', 'longitude': 'lon', 'crop_yield': crop_type})

In [143]:
maize_csvs[0]

,lat,lon,maize,growing_season
0,-89.75,0.25,NaN,25.0
1,-89.75,0.75,NaN,25.0
2,-89.75,1.25,NaN,25.0
3,-89.75,1.75,NaN,25.0
4,-89.75,2.25,NaN,25.0
...,...,...,...,...
9331195,89.75,357.75,NaN,27.0
9331196,89.75,358.25,NaN,27.0
9331197,89.75,358.75,NaN,27.0
9331198,89.75,359.25,NaN,27.0


In [145]:
for i in range(0, len(maize_csvs)):
  historical_df = historical_df.merge(maize_csvs[i], on = ['lat', 'lon', 'growing_season'], how = 'left')

In [152]:
save_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/train_data/'
historical_df.to_csv(save_path + 'historical_df.csv')

## ssp126

In [ ]:
#get all x features in one df
scenario = 'ssp126'
ssp126_df = scenarios[scenario][starting_variable]
for variable in variables:
  ssp126_df = ssp126_df.merge(scenarios[scenario][variable].drop('scenario', axis=1), on = ['lat', 'lon', 'growing_season'], how = 'left')
ssp126_df
print(ssp126_df.isna().sum())

In [153]:
save_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/train_data/'
ssp126_df.to_csv(save_path + 'ssp126_df.csv')

## ssp370

In [ ]:
#get all x features in one df
scenario = 'ssp370'
ssp370_df = scenarios[scenario][starting_variable]
for variable in variables:
  ssp370_df = ssp370_df.merge(scenarios[scenario][variable].drop('scenario', axis=1), on = ['lat', 'lon', 'growing_season'], how = 'left')
ssp370_df
print(ssp370_df.isna().sum())

In [157]:
save_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/train_data/'
ssp370_df.to_csv(save_path + 'ssp370_df.csv')

## ssp585

In [ ]:
#get all x features in one df
scenario = 'ssp585'
ssp585_df = scenarios[scenario][starting_variable]
for variable in variables:
  ssp585_df = ssp585_df.merge(scenarios[scenario][variable].drop('scenario', axis=1), on = ['lat', 'lon', 'growing_season'], how = 'left')
ssp585_df
print(ssp585_df.isna().sum())

In [158]:
save_path = '/content/drive/Shareddrives/Spatial_Term_Paper/data/train_data/'
ssp585_df.to_csv(save_path + 'ssp585_df.csv')